In [2]:
import pandas as pd
import os
# data_dirpath = '/usr2/mamille2/tumblr/data' # erebor
data_dirpath = '/usr0/home/mamille2/tumblr/data' # misty

# See if can use data that already have for reblog prediction

## Load followers

In [2]:
follower_data = pd.read_pickle(os.path.join(data_dirpath, 'blog_descriptions_recent100.pkl'))
print(len(follower_data))
print(follower_data.columns)

5226750
Index(['tumblog_id', 'activity_time_epoch', 'tumblr_blog_name',
       'tumblr_blog_title', 'tumblr_blog_description', 'tumblr_blog_url',
       'tumblr_blog_theme', 'is_group_blog', 'is_primary', 'is_private',
       'created_time_epoch', 'updated_time_epoch', 'timezone', 'language',
       'blog_classifier', 'generated_date', 'parsed_blog_description',
       'age_terms', 'age', 'ethnicity/nationality_terms',
       'ethnicity/nationality', 'fandoms_terms', 'fandoms', 'gender_terms',
       'gender', 'gender/sexuality_terms', 'gender/sexuality',
       'interests_terms', 'interests', 'location_terms', 'location',
       'personality type_terms', 'personality type', 'pronouns_terms',
       'pronouns', 'relationship status_terms', 'relationship status',
       'roleplay_terms', 'roleplay', 'roleplay/fandoms_terms',
       'roleplay/fandoms', 'sexual orientation_terms', 'sexual orientation',
       'weight_terms', 'weight', 'ethnicity/nationality_hegemonic_present',
       'gen

## Load followees

In [2]:
follow_data = pd.read_pickle(os.path.join(data_dirpath, 'follow_data_recent100.pkl'))
print(len(follow_data))
print(follow_data.columns)

329702060
Index(['tumblog_id', 'followed_tumblog_id', 'activity_time_epoch'], dtype='object')


## Get followees (now a script)

In [11]:
# Test follow structure

follow_fpath = '/usr2/kmaki/tumblr/follow_network.tsv' # 99G
chunksize = int(1e5)
hdr = ['tumblog_id', 'followed_tumblog_id', 'activity_time_epoch']
chunked = pd.read_csv(follow_fpath, sep='\t', header=None, names=hdr, iterator=True)
follow_data = chunked.get_chunk(chunksize)
print(len(follow_data))
print(follow_data.columns)

100000
Index(['tumblog_id', 'followed_tumblog_id', 'activity_time_epoch'], dtype='object')


In [3]:
# See what follow structure can find for set of followers
from tqdm import tqdm_notebook as tqdm
import numpy as np

follow_fpath = '/usr2/kmaki/tumblr/follow_network.tsv' # 99G
chunksize = int(1e5)
total_iters = int(np.ceil(3220088178 / chunksize))
hdr = ['tumblog_id', 'followed_tumblog_id', 'activity_time_epoch']
chunked = pd.read_csv(follow_fpath, sep='\t', header=None, names=hdr, chunksize=chunksize)

stx_follower_ids = set()
# follower_ids = follower_data['tumblog_id'].tolist()
# followee_data = []

for chunk in tqdm(chunked, total=total_iters):
    stx_follower_ids |= set(chunk['tumblog_id'].unique())
#     followee_data.append(chunk[chunk['tumblog_id'].isin(follower_ids)].values)
    
    
print(len(stx_follower_ids))

follower_ids = stx_follower_ids.intersection(set(follower_data['tumblog_id']))
len(follower_ids) # Have at least one piece of follow information for these users


100861970


3044188

In [5]:
# Save out follower ids
with open(os.path.join(data_dirpath, 'halfday_followers_with_descriptions.txt'), 'w') as f:
    for i in sorted(follower_ids):
        f.write(f'{i}\n')

In [4]:
# Get followee data
follow_fpath = '/usr2/kmaki/tumblr/follow_network.tsv' # 99G
chunksize = int(1e5)
total_iters = int(np.ceil(3220088178 / chunksize))
hdr = ['tumblog_id', 'followed_tumblog_id', 'activity_time_epoch']
chunked = pd.read_csv(follow_fpath, sep='\t', header=None, names=hdr, chunksize=chunksize)

followee_data = [] # list of np arrays to be concatenated

for chunk in tqdm(chunked, total=total_iters):
    followee_data.append(chunk[chunk['tumblog_id'].isin(follower_ids)].values)
    
follower_follow_data = pd.DataFrame(np.vstack(followee_data), columns=hdr)
len(follower_follow_data)

KeyboardInterrupt: 

In [29]:
# Save follow data from followers for which have descriptions
out_fname = 'follow_data_recent100.pkl'

follower_follow_data.to_pickle(os.path.join(data_dirpath, out_fname))

## Get followee descriptions

In [3]:
# Load blog descriptions
descs = pd.read_pickle(os.path.join(data_dirpath, 'blog_descriptions_recent100.pkl'))
print(len(descs))
print(descs.columns)

5226750
Index(['tumblog_id', 'activity_time_epoch', 'tumblr_blog_name',
       'tumblr_blog_title', 'tumblr_blog_description', 'tumblr_blog_url',
       'tumblr_blog_theme', 'is_group_blog', 'is_primary', 'is_private',
       'created_time_epoch', 'updated_time_epoch', 'timezone', 'language',
       'blog_classifier', 'generated_date', 'parsed_blog_description',
       'age_terms', 'age', 'ethnicity/nationality_terms',
       'ethnicity/nationality', 'fandoms_terms', 'fandoms', 'gender_terms',
       'gender', 'gender/sexuality_terms', 'gender/sexuality',
       'interests_terms', 'interests', 'location_terms', 'location',
       'personality type_terms', 'personality type', 'pronouns_terms',
       'pronouns', 'relationship status_terms', 'relationship status',
       'roleplay_terms', 'roleplay', 'roleplay/fandoms_terms',
       'roleplay/fandoms', 'sexual orientation_terms', 'sexual orientation',
       'weight_terms', 'weight', 'ethnicity/nationality_hegemonic_present',
       'gen

In [5]:
followees = set(follow_data['followed_tumblog_id'])
print(len(followees))

followees_with_descriptions = followees.intersection(set(descs['tumblog_id']))
print(len(followees_with_descriptions))

11816219
1454160


##  Select posts from followees, posted after followers followed them

In [6]:
# Load post data

post_fname = 'textposts_recent100.pkl' # ~75 GB in memory
post_data = pd.read_pickle(os.path.join(data_dirpath, post_fname))
print(len(post_data))
print(post_data.columns)

27342192
Index(['post_id', 'activity_time_epoch', 'tumblog_id', 'post_title',
       'post_short_url', 'post_type', 'post_caption', 'post_format',
       'post_note_count', 'created_time_epoch', 'updated_time_epoch',
       'is_submission', 'source_title', 'source_url', 'post_classifier',
       'blog_classifier', 'accepts_answers', 'reblogged_from_post_id',
       'reblogged_from_metadata', 'root_post_id', 'body', 'mentions',
       'post_tags', 'parsed_tags_minfreq1'],
      dtype='object')


In [8]:
followee_posts = post_data[post_data['tumblog_id'].isin(followees_with_descriptions)] # all posts regardless of time posted
print(len(followee_posts))

14852224


In [9]:
# Save followee posts
followee_posts.to_pickle(os.path.join(data_dirpath, 'followee_posts_recent100.pkl'))

In [3]:
# Load followee posts
# followee_posts = pd.read_pickle(os.path.join(data_dirpath, 'followee_posts_recent100.pkl'))
followee_posts = pd.read_pickle(os.path.join(data_dirpath, 'reblog_followee_posts_recent100.pkl'))
print(len(followee_posts))
print(followee_posts.columns)

11689157
Index(['post_id', 'activity_time_epoch', 'tumblog_id', 'post_title',
       'post_short_url', 'post_type', 'post_caption', 'post_format',
       'post_note_count', 'created_time_epoch', 'updated_time_epoch',
       'is_submission', 'source_title', 'source_url', 'post_classifier',
       'blog_classifier', 'accepts_answers', 'reblogged_from_post_id',
       'reblogged_from_metadata', 'root_post_id', 'body', 'mentions',
       'post_tags', 'parsed_tags_minfreq1'],
      dtype='object')


### Merge text and user info for quick prediction (duplicate posts for followees with multiple followers in the dataset)
Would filter followee posts based on timestamp of follows here

#### Make a dictionary of followees -> followers

In [3]:
# follow_data = pd.read_pickle(os.path.join(data_dirpath, 'follow_data_recent100.pkl'))
follow_data = pd.read_pickle(os.path.join(data_dirpath, 'reblog_follow_data_recent100.pkl'))
print(len(follow_data))
print(follow_data.columns)

329702060
Index(['tumblog_id', 'followed_tumblog_id', 'activity_time_epoch'], dtype='object')


In [43]:
gped = follow_data.set_index('tumblog_id').groupby('followed_tumblog_id')
gped.groups

len(gped.groups)

follow_dict = {key: list(gped.groups[key]) for key in gped.groups}
print(len(follow_dict))

8591392


In [25]:
# Average #followers/follower
len([d for l in follow_dict.values() for d in l]) / len(follow_dict)

27.90250079149684

In [44]:
# Save
import pickle

# with open(os.path.join(data_dirpath, 'follow_recent100_dict.pkl'), 'wb') as f:
with open(os.path.join(data_dirpath, 'reblog_follow_recent100_dict.pkl'), 'wb') as f:
    pickle.dump(follow_dict, f)

In [4]:
# Load follow dict
import pickle

# with open(os.path.join(data_dirpath, 'follow_recent100_dict.pkl'), 'wb') as f:
# followee: [followers]
with open(os.path.join(data_dirpath, 'reblog_follow_recent100_dict.pkl'), 'rb') as f: # followers are all rebloggers
    follow_dict = pickle.load(f)

In [6]:
# Sample from posts

# posts = followee_posts.sample(int(1e4))
posts = followee_posts.sample(int(1e5))
print(len(posts))
posts.columns

100000


Index(['post_id', 'activity_time_epoch', 'tumblog_id', 'post_title',
       'post_short_url', 'post_type', 'post_caption', 'post_format',
       'post_note_count', 'created_time_epoch', 'updated_time_epoch',
       'is_submission', 'source_title', 'source_url', 'post_classifier',
       'blog_classifier', 'accepts_answers', 'reblogged_from_post_id',
       'reblogged_from_metadata', 'root_post_id', 'body', 'mentions',
       'post_tags', 'parsed_tags_minfreq1'],
      dtype='object')

In [9]:
# Merge posts with follower/followee names, sample max 10 followers
import random
from tqdm import tqdm_notebook as tqdm

def safe_sample(l, t):
    if len(l) > t:
        return random.sample(l, t)
    else:
        return l

# posts['followers'] = posts['tumblog_id'].map(lambda x: safe_sample(follow_dict[x], 10))
posts['followers'] = posts['tumblog_id'].map(lambda x: follow_dict[x]) # All followers should be rebloggers
# posts['followers'] = list(map(lambda x: [u for u in follow_dict[x] if u in reblog_followers], tqdm(posts['tumblog_id']))) # Keep reblog followers

s = posts.apply(lambda x: pd.Series(x['followers']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'follower'
data = posts.join(s)
data.reset_index(drop=True, inplace=True)
data.drop('followers', axis=1, inplace=True)

print(len(data))

2703854


#### Merge in follower, followee blog description info

In [10]:
# Load blog descriptions
descs = pd.read_pickle(os.path.join(data_dirpath, 'blog_descriptions_recent100.pkl')) # Old data--no identity labels
print(len(descs))
print(descs.columns)

5238440
Index(['tumblog_id', 'activity_time_epoch', 'tumblr_blog_name',
       'tumblr_blog_title', 'tumblr_blog_description', 'tumblr_blog_url',
       'tumblr_blog_theme', 'is_group_blog', 'is_primary', 'is_private',
       'created_time_epoch', 'updated_time_epoch', 'timezone', 'language',
       'blog_classifier', 'generated_date', 'parsed_blog_description'],
      dtype='object')


In [11]:
# Merge in followee info
merged = pd.merge(data, descs, on='tumblog_id', how='left', suffixes=('_post', '_desc'))
print(len(merged))
print(merged.columns)

2703854
Index(['post_id', 'activity_time_epoch_post', 'tumblog_id', 'post_title',
       'post_short_url', 'post_type', 'post_caption', 'post_format',
       'post_note_count', 'created_time_epoch_post', 'updated_time_epoch_post',
       'is_submission', 'source_title', 'source_url', 'post_classifier',
       'blog_classifier_post', 'accepts_answers', 'reblogged_from_post_id',
       'reblogged_from_metadata', 'root_post_id', 'body', 'mentions',
       'post_tags', 'parsed_tags_minfreq1', 'follower',
       'activity_time_epoch_desc', 'tumblr_blog_name', 'tumblr_blog_title',
       'tumblr_blog_description', 'tumblr_blog_url', 'tumblr_blog_theme',
       'is_group_blog', 'is_primary', 'is_private', 'created_time_epoch_desc',
       'updated_time_epoch_desc', 'timezone', 'language',
       'blog_classifier_desc', 'generated_date', 'parsed_blog_description'],
      dtype='object')


In [12]:
# Merge in follower info
merged = pd.merge(merged, descs, left_on='follower', right_on='tumblog_id', how='left', suffixes=('_followee', '_follower'))
print(len(merged))
print(merged.columns)

2703854
Index(['post_id', 'activity_time_epoch_post', 'tumblog_id_followee',
       'post_title', 'post_short_url', 'post_type', 'post_caption',
       'post_format', 'post_note_count', 'created_time_epoch_post',
       'updated_time_epoch_post', 'is_submission', 'source_title',
       'source_url', 'post_classifier', 'blog_classifier_post',
       'accepts_answers', 'reblogged_from_post_id', 'reblogged_from_metadata',
       'root_post_id', 'body', 'mentions', 'post_tags', 'parsed_tags_minfreq1',
       'follower', 'activity_time_epoch_desc', 'tumblr_blog_name_followee',
       'tumblr_blog_title_followee', 'tumblr_blog_description_followee',
       'tumblr_blog_url_followee', 'tumblr_blog_theme_followee',
       'is_group_blog_followee', 'is_primary_followee', 'is_private_followee',
       'created_time_epoch_desc', 'updated_time_epoch_desc',
       'timezone_followee', 'language_followee', 'blog_classifier_desc',
       'generated_date_followee', 'parsed_blog_description_followee',


In [13]:
# Save out merged doc
# merged.to_pickle(os.path.join(data_dirpath, 'posts_descs_54k.pkl'))
merged.to_pickle(os.path.join(data_dirpath, 'posts_descs_3m_rebloggers.pkl'))

## Get outcome measure: whether or not a post was reblogged

In [3]:
# Load post data

post_fname = 'textposts_recent100.pkl' # ~75 GB in memory
post_data = pd.read_pickle(os.path.join(data_dirpath, post_fname))
print(len(post_data))
print(post_data.columns)

27342192
Index(['post_id', 'activity_time_epoch', 'tumblog_id', 'post_title',
       'post_short_url', 'post_type', 'post_caption', 'post_format',
       'post_note_count', 'created_time_epoch', 'updated_time_epoch',
       'is_submission', 'source_title', 'source_url', 'post_classifier',
       'blog_classifier', 'accepts_answers', 'reblogged_from_post_id',
       'reblogged_from_metadata', 'root_post_id', 'body', 'mentions',
       'post_tags', 'parsed_tags_minfreq1'],
      dtype='object')


In [5]:
# Load data to have outcome added to (full)
followee_posts = pd.read_pickle(os.path.join(data_dirpath, 'followee_posts_recent100.pkl'))
print(len(followee_posts))
print(followee_posts.columns)

14852224
Index(['post_id', 'activity_time_epoch', 'tumblog_id', 'post_title',
       'post_short_url', 'post_type', 'post_caption', 'post_format',
       'post_note_count', 'created_time_epoch', 'updated_time_epoch',
       'is_submission', 'source_title', 'source_url', 'post_classifier',
       'blog_classifier', 'accepts_answers', 'reblogged_from_post_id',
       'reblogged_from_metadata', 'root_post_id', 'body', 'mentions',
       'post_tags', 'parsed_tags_minfreq1'],
      dtype='object')


In [4]:
# Load data to have outcome added to (sampled)
# data = pd.read_pickle(os.path.join(data_dirpath, 'posts_descs_54k.pkl'))
data = pd.read_pickle(os.path.join(data_dirpath, 'posts_descs_3m_rebloggers.pkl'))
print(len(data))
print(data.columns)

2703854
Index(['post_id', 'activity_time_epoch_post', 'tumblog_id_followee',
       'post_title', 'post_short_url', 'post_type', 'post_caption',
       'post_format', 'post_note_count', 'created_time_epoch_post',
       ...
       'gender_hegemonic_present_follower',
       'pronouns_hegemonic_present_follower',
       'sexual_orientation_hegemonic_present_follower',
       'ethnicity/nationality_opposite_present_follower',
       'gender_opposite_present_follower',
       'pronouns_opposite_present_follower',
       'sexual_orientation_opposite_present_follower',
       'gender/sexuality_opposite_present_follower',
       'gender/sexuality_hegemonic_present_follower', 'post_text'],
      dtype='object', length=135)


In [5]:
print(post_data['reblogged_from_post_id'].dtype)
print(data['post_id'].dtype)

object
object


In [6]:
post_data['reblogged_from_post_id'] = post_data['reblogged_from_post_id'].fillna(-1).astype(int)
print(post_data['reblogged_from_post_id'].dtype)

reblogged_posts = post_data['reblogged_from_post_id'].unique()
print(len(reblogged_posts))

data['post_id'] = data['post_id'].astype(int)
print(data['post_id'].dtype)
posts = data['post_id'].unique()
print(len(posts))

int64
12102812
int64
99937


In [7]:
# How many posts from followees were reblogged by anybody in our data?

intersect = set(posts).intersection(set(reblogged_posts))
len(intersect)

6296

In [8]:
# Just consider opportunities for posts that were reblogged
reblog_data = data[data['post_id'].isin(intersect)]
len(reblog_data)

811256

In [9]:
# Any reblogs by followers? 

# Construct set of tuples (post_id, tumblog_id_follower) to search through
reblog_tuples = set(zip(post_data['reblogged_from_post_id'], post_data['tumblog_id']))
len(reblog_tuples)

20484021

In [10]:
# Any reblogs by followers? -- Just look at intersection as candidates for reblogs
from IPython.core.debugger import set_trace
from tqdm import tqdm_notebook as tqdm

# def is_reblog(x):
#     rows = post_data[(post_data['reblogged_from_post_id'] == x['post_id']) & \
#                                   (post_data['tumblog_id'] == x['tumblog_id_follower'])]
# data['reblogged'] = data.apply(is_reblog, axis=1)

def is_reblog(post_id, tumblog_id_follower):
    
    return (post_id, tumblog_id_follower) in reblog_tuples

# data['reblogged'] = [is_reblog(post_id, tumblog_id_follower) for post_id, tumblog_id_follower \
#                      in tqdm(zip(data['post_id'], data['tumblog_id_follower']), total=len(data))]
reblog_data['reblogged'] = [is_reblog(post_id, tumblog_id_follower) for post_id, tumblog_id_follower \
                     in tqdm(zip(reblog_data['post_id'], reblog_data['tumblog_id_follower']), total=len(reblog_data))]

sum(reblog_data['reblogged'])

/usr0/home/mamille2/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


2287

In [12]:
merged = data.join(reblog_data['reblogged'])
merged['reblogged']

merged['reblogged'] = merged['reblogged'].fillna(False)
sum(merged['reblogged'])

2287

In [13]:
merged.to_pickle(os.path.join(data_dirpath, 'posts_descs_3m_rebloggers.pkl'))

## Downsample to just followers who have reblogged at least once
More restrictive: users who have reblogged and we have text data for them

In [ ]:
# Load post data

post_fname = 'textposts_recent100.pkl' # ~75 GB in memory
post_data = pd.read_pickle(os.path.join(data_dirpath, post_fname))
print(len(post_data))
print(post_data.columns)

In [31]:
# Find list of users who have reblogged at least once
reblogs = post_data[post_data['reblogged_from_post_id'] >= 0]
print(len(reblogs))

19855879


In [33]:
reblog_followers = set(reblogs['tumblog_id'].unique())
len(reblog_followers)

1843817

In [46]:
reblog_followers = reblog_users

In [30]:
followee_posts.columns

Index(['post_id', 'activity_time_epoch', 'tumblog_id', 'post_title',
       'post_short_url', 'post_type', 'post_caption', 'post_format',
       'post_note_count', 'created_time_epoch', 'updated_time_epoch',
       'is_submission', 'source_title', 'source_url', 'post_classifier',
       'blog_classifier', 'accepts_answers', 'reblogged_from_post_id',
       'reblogged_from_metadata', 'root_post_id', 'body', 'mentions',
       'post_tags', 'parsed_tags_minfreq1'],
      dtype='object')

In [35]:
follow_data = pd.read_pickle(os.path.join(data_dirpath, 'follow_data_recent100.pkl'))
print(len(follow_data))
print(follow_data.columns)

329702060
Index(['tumblog_id', 'followed_tumblog_id', 'activity_time_epoch'], dtype='object')


In [36]:
# Restrict followee posts to followees that have at least one follower who has reblogged at least one post

reblog_follow_data = follow_data[follow_data['tumblog_id'].isin(reblog_users)]
reblog_followees = set(reblog_follow_data['followed_tumblog_id'].unique())
print(len(reblog_followees))

8591392


In [37]:
len(follow_data['followed_tumblog_id'].unique())

11816219

In [38]:
# Save out reblog follow data
reblog_follow_data.to_pickle(os.path.join(data_dirpath, 'reblog_follow_data_recent100.pkl')) # followers are rebloggers

In [5]:
# Load reblog follow data
reblog_follow_data = pd.read_pickle(os.path.join(data_dirpath, 'reblog_follow_data_recent100.pkl'))
reblog_followees = set(reblog_follow_data['followed_tumblog_id'].unique())
print(len(reblog_followees))

8591392


In [6]:
# Filter followee posts to just those that have followers who have reblogged
reblog_followee_posts = followee_posts[followee_posts['tumblog_id'].isin(reblog_followees)]
print(len(reblog_followee_posts))
print(len(followee_posts))

11689157
14852224


In [8]:
# Save reblog followee posts
reblog_followee_posts.to_pickle(os.path.join(data_dirpath, 'reblog_followee_posts_recent100.pkl'))

# Add blog description annotations to 3M sample (one-time)

In [2]:
# Load blog descriptions
descs = pd.read_pickle(os.path.join(data_dirpath, 'blog_descriptions_recent100.pkl'))
print(len(descs))
print(descs.columns)

5226750
Index(['tumblog_id', 'activity_time_epoch', 'tumblr_blog_name',
       'tumblr_blog_title', 'tumblr_blog_description', 'tumblr_blog_url',
       'tumblr_blog_theme', 'is_group_blog', 'is_primary', 'is_private',
       'created_time_epoch', 'updated_time_epoch', 'timezone', 'language',
       'blog_classifier', 'generated_date', 'parsed_blog_description',
       'age_terms', 'age', 'ethnicity/nationality_terms',
       'ethnicity/nationality', 'fandoms_terms', 'fandoms', 'gender_terms',
       'gender', 'gender/sexuality_terms', 'gender/sexuality',
       'interests_terms', 'interests', 'location_terms', 'location',
       'personality type_terms', 'personality type', 'pronouns_terms',
       'pronouns', 'relationship status_terms', 'relationship status',
       'roleplay_terms', 'roleplay', 'roleplay/fandoms_terms',
       'roleplay/fandoms', 'sexual orientation_terms', 'sexual orientation',
       'weight_terms', 'weight', 'ethnicity/nationality_hegemonic_present',
       'gen

In [6]:
# Load data to have outcome added to (sampled)
# data = pd.read_pickle(os.path.join(data_dirpath, 'posts_descs_54k.pkl'))
data = pd.read_pickle(os.path.join(data_dirpath, 'posts_descs_3m_rebloggers.pkl'))
print(len(data))
print(data.columns)

2703854
Index(['post_id', 'activity_time_epoch_post', 'tumblog_id_followee',
       'post_title', 'post_short_url', 'post_type', 'post_caption',
       'post_format', 'post_note_count', 'created_time_epoch_post',
       'updated_time_epoch_post', 'is_submission', 'source_title',
       'source_url', 'post_classifier', 'blog_classifier_post',
       'accepts_answers', 'reblogged_from_post_id', 'reblogged_from_metadata',
       'root_post_id', 'body', 'mentions', 'post_tags', 'parsed_tags_minfreq1',
       'follower', 'activity_time_epoch_desc', 'tumblr_blog_name_followee',
       'tumblr_blog_title_followee', 'tumblr_blog_description_followee',
       'tumblr_blog_url_followee', 'tumblr_blog_theme_followee',
       'is_group_blog_followee', 'is_primary_followee', 'is_private_followee',
       'created_time_epoch_desc', 'updated_time_epoch_desc',
       'timezone_followee', 'language_followee', 'blog_classifier_desc',
       'generated_date_followee', 'parsed_blog_description_followee',


In [4]:
# Remove columns previously merged
merged_cols = [col for col in data.columns if not (col.endswith('_desc') or \
               col.endswith('_followee') or col.endswith('_follower'))]
data = data[merged_cols]
data.columns

Index(['post_id', 'activity_time_epoch_post', 'post_title', 'post_short_url',
       'post_type', 'post_caption', 'post_format', 'post_note_count',
       'created_time_epoch_post', 'updated_time_epoch_post', 'is_submission',
       'source_title', 'source_url', 'post_classifier', 'blog_classifier_post',
       'accepts_answers', 'reblogged_from_post_id', 'reblogged_from_metadata',
       'root_post_id', 'body', 'mentions', 'post_tags', 'parsed_tags_minfreq1',
       'follower', 'activity_time_epoch', 'created_time_epoch',
       'updated_time_epoch', 'blog_classifier'],
      dtype='object')

In [12]:
pprint(sorted(data.columns.tolist()))

['accepts_answers',
 'activity_time_epoch',
 'activity_time_epoch_desc',
 'activity_time_epoch_post',
 'blog_classifier',
 'blog_classifier_desc',
 'blog_classifier_post',
 'body',
 'created_time_epoch',
 'created_time_epoch_desc',
 'created_time_epoch_post',
 'follower',
 'generated_date_followee',
 'generated_date_follower',
 'is_group_blog_followee',
 'is_group_blog_follower',
 'is_primary_followee',
 'is_primary_follower',
 'is_private_followee',
 'is_private_follower',
 'is_submission',
 'language_followee',
 'language_follower',
 'mentions',
 'parsed_blog_description_followee',
 'parsed_blog_description_follower',
 'parsed_tags_minfreq1',
 'post_caption',
 'post_classifier',
 'post_format',
 'post_id',
 'post_note_count',
 'post_short_url',
 'post_tags',
 'post_title',
 'post_type',
 'reblogged_from_metadata',
 'reblogged_from_post_id',
 'root_post_id',
 'source_title',
 'source_url',
 'timezone_followee',
 'timezone_follower',
 'tumblog_id_followee',
 'tumblog_id_follower',
 'tu

In [13]:
data.rename(columns={
    'activity_time_epoch': 'activity_time_epoch_follower',
    'activity_time_epoch_desc': 'activity_time_epoch_followee',
    'blog_classifier': 'blog_classifier_follower',
    'blog_classifier_desc': 'blog_classifier_followee',
    'created_time_epoch': 'created_time_epoch_follower',
    'created_time_epoch_desc': 'created_time_epoch_followee',
    'updated_time_epoch': 'updated_time_epoch_follower',
    'updated_time_epoch_desc': 'updated_time_epoch_followee',
}, inplace=True)

pprint(sorted(data.columns.tolist()))

['accepts_answers',
 'activity_time_epoch_followee',
 'activity_time_epoch_follower',
 'activity_time_epoch_post',
 'blog_classifier_followee',
 'blog_classifier_follower',
 'blog_classifier_post',
 'body',
 'created_time_epoch_followee',
 'created_time_epoch_follower',
 'created_time_epoch_post',
 'follower',
 'generated_date_followee',
 'generated_date_follower',
 'is_group_blog_followee',
 'is_group_blog_follower',
 'is_primary_followee',
 'is_primary_follower',
 'is_private_followee',
 'is_private_follower',
 'is_submission',
 'language_followee',
 'language_follower',
 'mentions',
 'parsed_blog_description_followee',
 'parsed_blog_description_follower',
 'parsed_tags_minfreq1',
 'post_caption',
 'post_classifier',
 'post_format',
 'post_id',
 'post_note_count',
 'post_short_url',
 'post_tags',
 'post_title',
 'post_type',
 'reblogged_from_metadata',
 'reblogged_from_post_id',
 'root_post_id',
 'source_title',
 'source_url',
 'timezone_followee',
 'timezone_follower',
 'tumblog_id_

In [14]:
# Merge in followee info
merged = pd.merge(data, descs, left_on='tumblog_id_followee', right_on='tumblog_id', how='left', 
#                   suffixes=('_post', '_followee')
                 )
print(len(merged))
pprint(sorted(merged.columns.tolist()))

2703854
['accepts_answers',
 'activity_time_epoch',
 'activity_time_epoch_followee',
 'activity_time_epoch_follower',
 'activity_time_epoch_post',
 'age',
 'age_terms',
 'blog_classifier',
 'blog_classifier_followee',
 'blog_classifier_follower',
 'blog_classifier_post',
 'body',
 'created_time_epoch',
 'created_time_epoch_followee',
 'created_time_epoch_follower',
 'created_time_epoch_post',
 'ethnicity/nationality',
 'ethnicity/nationality_hegemonic_present',
 'ethnicity/nationality_opposite_present',
 'ethnicity/nationality_terms',
 'fandoms',
 'fandoms_terms',
 'follower',
 'gender',
 'gender/sexuality',
 'gender/sexuality_hegemonic_present',
 'gender/sexuality_opposite_present',
 'gender/sexuality_terms',
 'gender_hegemonic_present',
 'gender_opposite_present',
 'gender_terms',
 'generated_date',
 'generated_date_followee',
 'generated_date_follower',
 'interests',
 'interests_terms',
 'is_group_blog',
 'is_group_blog_followee',
 'is_group_blog_follower',
 'is_primary',
 'is_prima

In [15]:
merged.drop('activity_time_epoch', axis=1, inplace=True)
merged.drop('blog_classifier', axis=1, inplace=True)
merged.drop('created_time_epoch', axis=1, inplace=True)
merged.drop('generated_date', axis=1, inplace=True)
merged.drop('is_group_blog', axis=1, inplace=True)
merged.drop('is_primary', axis=1, inplace=True)
merged.drop('is_private', axis=1, inplace=True)
merged.drop('language', axis=1, inplace=True)
merged.drop('parsed_blog_description', axis=1, inplace=True)
merged.drop('tumblr_blog_description', axis=1, inplace=True)
merged.drop('tumblr_blog_name', axis=1, inplace=True)
merged.drop('tumblr_blog_theme', axis=1, inplace=True)
merged.drop('tumblr_blog_title', axis=1, inplace=True)
merged.drop('tumblr_blog_url', axis=1, inplace=True)
merged.drop('updated_time_epoch', axis=1, inplace=True)
pprint(sorted(merged.columns.tolist()))

['accepts_answers',
 'activity_time_epoch_followee',
 'activity_time_epoch_follower',
 'activity_time_epoch_post',
 'age',
 'age_terms',
 'blog_classifier_followee',
 'blog_classifier_follower',
 'blog_classifier_post',
 'body',
 'created_time_epoch_followee',
 'created_time_epoch_follower',
 'created_time_epoch_post',
 'ethnicity/nationality',
 'ethnicity/nationality_hegemonic_present',
 'ethnicity/nationality_opposite_present',
 'ethnicity/nationality_terms',
 'fandoms',
 'fandoms_terms',
 'follower',
 'gender',
 'gender/sexuality',
 'gender/sexuality_hegemonic_present',
 'gender/sexuality_opposite_present',
 'gender/sexuality_terms',
 'gender_hegemonic_present',
 'gender_opposite_present',
 'gender_terms',
 'generated_date_followee',
 'generated_date_follower',
 'interests',
 'interests_terms',
 'is_group_blog_followee',
 'is_group_blog_follower',
 'is_primary_followee',
 'is_primary_follower',
 'is_private_followee',
 'is_private_follower',
 'is_submission',
 'language_followee',
 

In [16]:
desc_cats = [
    'age',
    'ethnicity/nationality',
    'fandoms',
    'gender',
    'gender/sexuality',
    'interests',
    'location',
    'personality_type',
    'pronouns',
    'roleplay',
    'roleplay/fandoms',
    'sexual orientation',
    'weight',
]

In [19]:
for cat in desc_cats:
    merged.rename(columns={
        cat: f'{cat.replace(" ","_")}_followee',
        f'{cat}_terms': f'{cat.replace(" ","_")}_terms_followee',
    }, inplace=True)
        
    if f'{cat}_hegemonic_present' in merged.columns:
        merged.rename(columns={
            f'{cat}_hegemonic_present': f'{cat.replace(" ","_")}_hegemonic_present_followee',
            f'{cat}_opposite_present': f'{cat.replace(" ","_")}_opposite_present_followee',
        }, inplace=True)

pprint(sorted(merged.columns.tolist()))

['accepts_answers',
 'activity_time_epoch_followee',
 'activity_time_epoch_follower',
 'activity_time_epoch_post',
 'age_followee',
 'age_terms_followee',
 'blog_classifier_followee',
 'blog_classifier_follower',
 'blog_classifier_post',
 'body',
 'created_time_epoch_followee',
 'created_time_epoch_follower',
 'created_time_epoch_post',
 'ethnicity/nationality_followee',
 'ethnicity/nationality_hegemonic_present_followee',
 'ethnicity/nationality_opposite_present_followee',
 'ethnicity/nationality_terms_followee',
 'fandoms_followee',
 'fandoms_terms_followee',
 'follower',
 'gender/sexuality_followee',
 'gender/sexuality_hegemonic_present_followee',
 'gender/sexuality_opposite_present_followee',
 'gender/sexuality_terms_followee',
 'gender_followee',
 'gender_hegemonic_present_followee',
 'gender_opposite_present_followee',
 'gender_terms_followee',
 'generated_date_followee',
 'generated_date_follower',
 'interests_followee',
 'interests_terms_followee',
 'is_group_blog_followee',
 '

In [20]:
# Merge in follower info
merged = pd.merge(merged, descs, left_on='follower', right_on='tumblog_id', how='left',
#                   suffixes=('_followee', '_follower')
                 )
print(len(merged))
pprint(sorted(merged.columns))

2703854
['accepts_answers',
 'activity_time_epoch',
 'activity_time_epoch_followee',
 'activity_time_epoch_follower',
 'activity_time_epoch_post',
 'age',
 'age_followee',
 'age_terms',
 'age_terms_followee',
 'blog_classifier',
 'blog_classifier_followee',
 'blog_classifier_follower',
 'blog_classifier_post',
 'body',
 'created_time_epoch',
 'created_time_epoch_followee',
 'created_time_epoch_follower',
 'created_time_epoch_post',
 'ethnicity/nationality',
 'ethnicity/nationality_followee',
 'ethnicity/nationality_hegemonic_present',
 'ethnicity/nationality_hegemonic_present_followee',
 'ethnicity/nationality_opposite_present',
 'ethnicity/nationality_opposite_present_followee',
 'ethnicity/nationality_terms',
 'ethnicity/nationality_terms_followee',
 'fandoms',
 'fandoms_followee',
 'fandoms_terms',
 'fandoms_terms_followee',
 'follower',
 'gender',
 'gender/sexuality',
 'gender/sexuality_followee',
 'gender/sexuality_hegemonic_present',
 'gender/sexuality_hegemonic_present_followee'

In [21]:
merged.drop('activity_time_epoch', axis=1, inplace=True)
merged.drop('blog_classifier', axis=1, inplace=True)
merged.drop('created_time_epoch', axis=1, inplace=True)
merged.drop('generated_date', axis=1, inplace=True)
merged.drop('is_group_blog', axis=1, inplace=True)
merged.drop('is_primary', axis=1, inplace=True)
merged.drop('is_private', axis=1, inplace=True)
merged.drop('language', axis=1, inplace=True)
merged.drop('parsed_blog_description', axis=1, inplace=True)
merged.drop('timezone_x', axis=1, inplace=True)
merged.drop('timezone_y', axis=1, inplace=True)
merged.drop('tumblog_id_x', axis=1, inplace=True)
merged.drop('tumblog_id_y', axis=1, inplace=True)
merged.drop('tumblr_blog_description', axis=1, inplace=True)
merged.drop('tumblr_blog_name', axis=1, inplace=True)
merged.drop('tumblr_blog_theme', axis=1, inplace=True)
merged.drop('tumblr_blog_title', axis=1, inplace=True)
merged.drop('tumblr_blog_url', axis=1, inplace=True)
merged.drop('updated_time_epoch', axis=1, inplace=True)
pprint(sorted(merged.columns.tolist()))

['accepts_answers',
 'activity_time_epoch_followee',
 'activity_time_epoch_follower',
 'activity_time_epoch_post',
 'age',
 'age_followee',
 'age_terms',
 'age_terms_followee',
 'blog_classifier_followee',
 'blog_classifier_follower',
 'blog_classifier_post',
 'body',
 'created_time_epoch_followee',
 'created_time_epoch_follower',
 'created_time_epoch_post',
 'ethnicity/nationality',
 'ethnicity/nationality_followee',
 'ethnicity/nationality_hegemonic_present',
 'ethnicity/nationality_hegemonic_present_followee',
 'ethnicity/nationality_opposite_present',
 'ethnicity/nationality_opposite_present_followee',
 'ethnicity/nationality_terms',
 'ethnicity/nationality_terms_followee',
 'fandoms',
 'fandoms_followee',
 'fandoms_terms',
 'fandoms_terms_followee',
 'follower',
 'gender',
 'gender/sexuality',
 'gender/sexuality_followee',
 'gender/sexuality_hegemonic_present',
 'gender/sexuality_hegemonic_present_followee',
 'gender/sexuality_opposite_present',
 'gender/sexuality_opposite_present

In [22]:
for cat in desc_cats:
    merged.rename(columns={
        cat: f'{cat.replace(" ","_")}_follower',
        f'{cat}_terms': f'{cat.replace(" ","_")}_terms_follower',
    }, inplace=True)
        
    if f'{cat}_hegemonic_present' in merged.columns:
        merged.rename(columns={
            f'{cat}_hegemonic_present': f'{cat.replace(" ","_")}_hegemonic_present_follower',
            f'{cat}_opposite_present': f'{cat.replace(" ","_")}_opposite_present_follower',
        }, inplace=True)

pprint(sorted(merged.columns.tolist()))

['accepts_answers',
 'activity_time_epoch_followee',
 'activity_time_epoch_follower',
 'activity_time_epoch_post',
 'age_followee',
 'age_follower',
 'age_terms_followee',
 'age_terms_follower',
 'blog_classifier_followee',
 'blog_classifier_follower',
 'blog_classifier_post',
 'body',
 'created_time_epoch_followee',
 'created_time_epoch_follower',
 'created_time_epoch_post',
 'ethnicity/nationality_followee',
 'ethnicity/nationality_follower',
 'ethnicity/nationality_hegemonic_present_followee',
 'ethnicity/nationality_hegemonic_present_follower',
 'ethnicity/nationality_opposite_present_followee',
 'ethnicity/nationality_opposite_present_follower',
 'ethnicity/nationality_terms_followee',
 'ethnicity/nationality_terms_follower',
 'fandoms_followee',
 'fandoms_follower',
 'fandoms_terms_followee',
 'fandoms_terms_follower',
 'follower',
 'gender/sexuality_followee',
 'gender/sexuality_follower',
 'gender/sexuality_hegemonic_present_followee',
 'gender/sexuality_hegemonic_present_follo

In [23]:
merged.to_pickle(os.path.join(data_dirpath, 'posts_descs_3m_rebloggers.pkl'))